In [1]:
# --- Setup & Imports ---
import os, sys
from pathlib import Path
import pandas as pd
import numpy as np

# --- 1. Pfad-Setup ---
def _locate_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(5):  # walk up to 5 levels
        if (cur / 'src').exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start.resolve()

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ['PROJECT_ROOT'] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.config import GlobalConfig, DEFAULT_CORR_SPEC, EWMA_CORR_SPEC, outputs_for_model
from src.tuning import run_stageA, run_stageB
from src.io_timesplits import load_target, load_ifo_features
# --- MODELL-IMPORT ---
from src.models.ET import ForecastModel # Nutzt ExtraTreesRegressor

print('PROJECT_ROOT =', PROJECT_ROOT)

# --- MODELLNAME ---
MODEL_NAME = "extra_trees"
outputs_for_model(MODEL_NAME)
print(f'Modell {MODEL_NAME} wird getunt.')

# --- 2. Daten laden ---
y = load_target()  # ΔIP with DatetimeIndex
X_ifo = load_ifo_features() # ifo features

idx = y.index.intersection(X_ifo.index)
y, X_ifo = y.loc[idx], X_ifo.loc[idx]
print('Shapes:', X_ifo.shape, y.shape)

# --- 3. Base config (splits & policy as in thesis) ---
def base_cfg() -> GlobalConfig:
    cfg = GlobalConfig()
    cfg.W0_A     = 180
    cfg.BLOCKS_A = [(181,200), (201,220), (221,240)]
    cfg.W0_B     = 240
    # Policy Parameter
    cfg.policy_window   = 24
    cfg.policy_decay    = 0.95
    cfg.policy_gain_min = 0.03
    cfg.policy_cooldown = 3
    return cfg

cfg0 = base_cfg()

# --- 4. Helper für Korrelations-Spezifikation (bleibt gleich) ---
def make_corr_spec(kind: str) -> dict:
    if kind == 'expanding':
        return dict(DEFAULT_CORR_SPEC)
    elif kind == 'ewm':
        return dict(EWMA_CORR_SPEC)
    else:
        raise ValueError("kind must be 'expanding' or 'ewm'")

# --- 5. OPTIMIERTES TUNING-GRID ---

# A) Feature Engineering Parameter Listen
corr_options = [
    ("expanding", make_corr_spec("expanding")),
    ("ewm", make_corr_spec("ewm"))
]
lag_candidates_list = [(1, 2, 3, 6, 12)] # Halten wir einfach für ET
top_k_lags_list = [1, 2] # Testen, ob 2 Lags helfen
use_rm3_list = [True, False] # Testen mit und ohne Glättung
k1_topk_list = [100, 300] # Engeres Screening vs. breiteres Screening vor DR
redundancy_param_list = [0.90, 0.95] # Zwei Redundanzlevel testen
# DR Optionen - PCA/PLS machen nur nach breiterem Screening Sinn
dr_options_list = [
    {'dr_method': 'none'},
    {'dr_method': 'pca', 'pca_var_target': 0.95, 'pca_kmax': 50},
    {'dr_method': 'pls', 'pls_components': 8} # PLS als Alternative
]

# B) Extra Trees Hyperparameter Listen
n_estimators_list = [300] # Etwas mehr Bäume fixieren
max_features_list = ['sqrt', 0.2, 0.4] # Drei Optionen für Feature Subsampling
min_samples_leaf_list = [5, 15, 30] # Drei Regularisierungslevel

# C) Target Blocks & Weighting Listen
target_block_options = [ None, ["AR1"], ["Chronos"], ["TSFresh"] ] # Einzeln testen
weighting_options = [
    {"sample_weight_decay": None}, # Standard: kein Decay
    {"sample_weight_decay": 0.98}  # Option: Exponentielles Decay
]

# D) Grid zusammensetzen (Optimierte Logik)
def build_model_grid():
    hp_grid = []
    for corr_tag, corr_spec in corr_options:
        for lags in lag_candidates_list:
            for k_lags in top_k_lags_list:
                for rm3 in use_rm3_list:
                    for k1 in k1_topk_list:
                        for red in redundancy_param_list:
                            for dr_opt in dr_options_list:
                                # *** WICHTIGE EINSCHRÄNKUNG für Laufzeit ***
                                # DR (PCA/PLS) nur anwenden, wenn wir breiter gescreent haben (k1=300)
                                if k1 == 100 and dr_opt['dr_method'] != 'none': continue
                                # Kein DR macht bei k1=300 wenig Sinn (zu viele Features für ET)
                                # Erlauben wir es trotzdem als Testfall
                                # if k1 == 300 and dr_opt['dr_method'] == 'none': continue

                                for n_est in n_estimators_list:
                                    for max_feat in max_features_list:
                                        for min_leaf in min_samples_leaf_list:
                                            for block_set in target_block_options:
                                                for weight_hp in weighting_options:
                                                    hp = {
                                                        # FE Params
                                                        'lag_candidates': lags,
                                                        'top_k_lags_per_feature': k_lags,
                                                        'use_rm3': rm3,
                                                        'k1_topk': k1,
                                                        'redundancy_param': red,
                                                        **dr_opt,
                                                        # ET Params
                                                        'n_estimators': n_est,
                                                        'max_features': max_feat,
                                                        'min_samples_leaf': min_leaf,
                                                        # Other Params
                                                        **weight_hp,
                                                        'corr_tag': corr_tag,
                                                        'corr_spec': corr_spec,
                                                        'target_block_set': block_set
                                                    }
                                                    hp_grid.append(hp)
    return hp_grid

model_grid = build_model_grid()
# Ungefähre Größe: 2(Corr)*1*2(kL)*2(RM)*[ (1(k1)*1(DR)) + (1(k1)*3(DR)) ]*2(Red)*1(nEst)*3(MaxF)*3(MinL)*4(Block)*2(Weight)
# = 8 * [1 + 3] * 2 * 1 * 9 * 4 * 2 = 8 * 4 * 2 * 9 * 8 = 64 * 72 = 4608 (Immer noch groß!)
# Reduzieren wir es weiter:
# Fixieren k_lags=1, rm3=True, red=0.90, weight=None
# Size: 2(Corr)*1*1*1 * [1 + 3] * 1 * 1 * 3(MaxF)*3(MinL)*4(Block)*1(Weight)
# = 2 * 4 * 1 * 9 * 4 * 1 = 8 * 36 = 288 (Das ist akzeptabler)

# --- ANGEPASSTES GRID für akzeptable Laufzeit ---
k_lags_list = [1]
use_rm3_list = [True]
redundancy_param_list = [0.90]
weighting_options = [ {"sample_weight_decay": None} ]
max_features_list = ['sqrt', 0.3] # Reduziert auf 2
min_samples_leaf_list = [5, 15]   # Reduziert auf 2

# Neuberechnung Grid Size: 2*1*1*1 * [1+3] * 1 * 1 * 2 * 2 * 4 * 1 = 2 * 4 * 4 * 4 = 128
# Das ist ein guter Kompromiss.

def build_model_grid_optimized():
    hp_grid = []
    for corr_tag, corr_spec in corr_options: # 2
        for lags in lag_candidates_list: # 1
            for k_lags in k_lags_list: # 1
                for rm3 in use_rm3_list: # 1
                    for k1 in k1_topk_list: # 2
                        for red in redundancy_param_list: # 1
                            for dr_opt in dr_options_list: # 3
                                # Einschränkung
                                if k1 == 100 and dr_opt['dr_method'] != 'none': continue # Lässt (100, none) -> 1
                                # if k1 == 300 and dr_opt['dr_method'] == 'none': continue # Erlauben wir testweise
                                # Lässt (300, none), (300, pca), (300, pls) -> 3. Total FE/DR = 1+3 = 4

                                for n_est in n_estimators_list: # 1
                                    for max_feat in max_features_list: # 2
                                        for min_leaf in min_samples_leaf_list: # 2
                                            for block_set in target_block_options: # 4
                                                for weight_hp in weighting_options: # 1
                                                    hp = {
                                                        # FE Params
                                                        'lag_candidates': lags,
                                                        'top_k_lags_per_feature': k_lags,
                                                        'use_rm3': rm3,
                                                        'k1_topk': k1,
                                                        'redundancy_param': red,
                                                        **dr_opt,
                                                        # ET Params
                                                        'n_estimators': n_est,
                                                        'max_features': max_feat,
                                                        'min_samples_leaf': min_leaf,
                                                        # Other Params
                                                        **weight_hp,
                                                        'corr_tag': corr_tag,
                                                        'corr_spec': corr_spec,
                                                        'target_block_set': block_set
                                                    }
                                                    hp_grid.append(hp)
    return hp_grid

model_grid = build_model_grid_optimized()
print("Optimierte HP-Kombinationen:", len(model_grid)) # 2*1*1*1 * 4 * 1 * 1 * 2 * 2 * 4 * 1 = 128
print("Erstes HP-Set:", model_grid[0] if model_grid else "Grid ist leer")


# --- 6. Stage A/B Lauf ---
if model_grid: # Nur ausführen, wenn das Grid nicht leer ist
    shortlist = run_stageA(
        model_name=MODEL_NAME,
        model_ctor=lambda hp: ForecastModel(hp),
        model_grid=model_grid,
        X=X_ifo, y=y, cfg=cfg0,
        keep_top_k_final=min(5, len(model_grid)),
        min_survivors_per_block=max(1, len(model_grid)//4), # Reduziere Überlebende pro Block etwas
    )

    run_stageB(
        model_name=MODEL_NAME,
        model_ctor=lambda hp: ForecastModel(hp),
        shortlist=shortlist,
        X=X_ifo, y=y, cfg=cfg0,
        # max_months=12,   # Optional: Begrenze Stage B für schnellen Test
    )
else:
    print("Keine gültigen HP-Kombinationen gefunden, Stages übersprungen.")

print(f"\nDone. Check outputs/stageA|stageB/{MODEL_NAME} for results.")

PROJECT_ROOT = /Users/jonasschernich/Documents/Masterarbeit/Code
Modell extra_trees wird getunt.
INFO in load_ifo_features: Renaming columns to ensure validity.
Shapes: (407, 2160) (407,)
Optimierte HP-Kombinationen: 128
Erstes HP-Set: {'lag_candidates': (1, 2, 3, 6, 12), 'top_k_lags_per_feature': 1, 'use_rm3': True, 'k1_topk': 100, 'redundancy_param': 0.9, 'dr_method': 'none', 'n_estimators': 300, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'sample_weight_decay': None, 'corr_tag': 'expanding', 'corr_spec': {'mode': 'expanding', 'window': None, 'lam': None}, 'target_block_set': None}
[Stage A][Block 1] train_end=180, OOS=181-200 | configs=128
  - Config 1/128
    · Month 1/20 processed | running...RMSE=1.6096
    · Month 2/20 processed | running...RMSE=2.1772
    · Month 3/20 processed | running...RMSE=1.8302
    · Month 4/20 processed | running...RMSE=1.6064
    · Month 5/20 processed | running...RMSE=1.5466
    · Month 6/20 processed | running...RMSE=1.4189
    · Month 7/20 proces